## Getting Started

### Install required libraries

In [9]:
%%capture

!sudo apt install -q jq

you will need to restart your kernal after running the first cell

### Set Google Cloud project

To get started using Vertex AI, the organizers will provide you with these parameters to connect to the API

In [10]:
# Define project information
PROJECT_ID = "oa-bta-learning-dv"  # @param {type:"string"}
LOCATION = "europe-west4"  # @param {type:"string"}

# Import libraries
import os
from google import genai


## Use the Gemini 2.0 Flash model

In [11]:
MODEL_ID = "gemini-2.0-flash"

## Text generation

The `generate_content` method can handle a wide variety of use cases, including multi-turn chat and multimodal input, depending on what the underlying model supports. In this example, you send a text prompt and request the model response in text.

In [ ]:
client = genai.Client(
    vertexai=True, project=PROJECT_ID, location=LOCATION
)
response = client.models.generate_content(
    model=MODEL_ID, contents="Tell me about data science"
)
print(response.text)

Okay, let's dive into the world of data science!  Here's a comprehensive overview of what it is, what it involves, and why it's such a hot field right now:

**What is Data Science?**

At its core, data science is an **interdisciplinary field** that uses scientific methods, processes, algorithms, and systems to **extract knowledge and insights from structured and unstructured data**.  Think of it as a blend of statistics, computer science, and domain expertise, all aimed at making better decisions and solving complex problems.

Here's a breakdown of the key elements:

*   **Data:** The foundation.  This can be anything from numerical data in spreadsheets to text documents, images, audio, videos, sensor data, social media posts, and much, much more.  The key is that it's recorded information.
*   **Science:**  Applying the scientific method – forming hypotheses, testing them with data, and drawing conclusions based on evidence.
*   **Interdisciplinary:** Data science pulls from various f

### Model parameters

Every prompt you send to the model includes parameter values that control how the model generates a response. The model can generate different results for different parameter values. You can experiment with different model parameters to see how the results change.

In [14]:
from google.genai import types

response = client.models.generate_content(
  model=MODEL_ID,
  contents='hello',
  config=types.GenerateContentConfig(
    system_instruction='talk like a pirate',
    max_output_tokens=3,
    temperature=0.3,
    response_logprobs=True,
    logprobs=3,
  ),
)
print(response.text)

Ahoy there


You can also set some safety settings for a safe user friendly discussion

In [16]:
from google.genai import types

response = client.models.generate_content(
  model='gemini-2.0-flash-001',
  contents='Say something bad.',
  config=types.GenerateContentConfig(
    safety_settings=[
      types.SafetySetting(
        category='HARM_CATEGORY_HATE_SPEECH',
        threshold='BLOCK_ONLY_HIGH',
      )
    ]
  ),
)
print(response.text)

I am programmed to be a harmless AI assistant. I cannot generate responses that are harmful, unethical, or promote discrimination.



### Chat

The Gemini API supports natural multi-turn conversations and is ideal for text tasks that require back-and-forth interactions.

In [17]:
chat = client.chats.create(model='gemini-2.0-flash-001')
response = chat.send_message('tell me a story')
print(response.text)
response = chat.send_message('summarize the story you told me in 1 sentence')
print(response.text)

The wind howled a mournful tune through the skeletal branches of the Whispering Woods, a sound Elara knew well. She huddled deeper into the worn, patchwork cloak her grandmother had made, the scent of lavender and woodsmoke a small comfort against the biting chill. For twelve years, Elara had lived on the edge of these woods, an orphan raised by the eccentric old woman known as Willow Weaver.

Willow Weaver wasn't just any old woman. She spoke to the trees, bartered with the moon, and brewed potions that smelled of sunshine and secrets. But now, Willow Weaver was gone, taken by a sudden illness that left Elara alone with only the whispering trees for company.

Willow Weaver had left her one thing: a small, wooden box tied with a faded green ribbon. Inside, nestled on a bed of velvet moss, was a single, iridescent feather. "When you are lost, Elara," Willow Weaver had rasped on her deathbed, "seek the Skylark's Song. It will guide you."

Elara clutched the feather, her heart heavy with 

## Multimodal input

Gemini is a multimodal model that supports adding image and video in text or chat prompts for a text response.


### Download an image from Google Cloud Storage

In [8]:
! gsutil cp "gs://cloud-samples-data/generative-ai/image/320px-Felis_catus-cat_on_snow.jpg" ./image.jpg

Copying gs://cloud-samples-data/generative-ai/image/320px-Felis_catus-cat_on_snow.jpg...
/ [0 files][    0.0 B/ 17.4 KiB]                                                
/ [0 files][ 17.4 KiB/ 17.4 KiB]                                                
-
- [1 files][ 17.4 KiB/ 17.4 KiB]                                                

Operation completed over 1 objects/17.4 KiB.                                     


### Generate text from a local image

Include the prompt and the `mime_type` field. The supported [MIME types](https://en.wikipedia.org/wiki/Media_type) for images include `image/png`, `image/jpeg` and `video/mp4`.

In this example we will use the model to describe the content of a video

In [22]:
from google import genai
from google.genai.types import HttpOptions, Part

response = client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents=[
        Part.from_uri(
            file_uri="gs://cloud-samples-data/generative-ai/video/ad_copy_from_video.mp4",
            mime_type="video/mp4",
        ),
        "What is in the video?",
    ],
)
print(response.text)

Here's a description of what's in the video:

The video captures an aerial view of surfers riding waves in the ocean. The water is a muted, deep blue-grey, and the sky is overcast, creating a soft, diffused light. In the background, there's a coastline with mountains obscured by mist or low-lying clouds.

Initially, there are several surfers visible: one riding a wave close to the camera, and others further out, some paddling or waiting. As the video progresses, the surfer closest to the camera maneuvers skillfully on the wave, making sharp turns and showcasing their surfing abilities. The wave eventually breaks, but the surfer continues to ride it toward the shoreline. Other surfers are seen paddling around, waiting for their turn to catch a wave. The camera maintains a wide shot throughout, keeping the surfer and the surrounding ocean in view.


### Generate text from an image on Google Cloud Storage

Specify the Cloud Storage URI of the image to include in the prompt. The bucket that stores the file must be in the same Google Cloud project that's sending the request. You must also specify the `mime_type` field. The supported image MIME types include `image/png` and `image/jpeg`.

### Generate text from a video file

Specify the Cloud Storage URI of the video to include in the prompt. The bucket that stores the file must be in the same Google Cloud project that's sending the request. You must also specify the `mime_type` field. The supported MIME types for video include `video/mp4`.


In [ ]:
from google import genai
from google.genai.types import HttpOptions, Part

response = client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents=[
        Part.from_uri(
            file_uri="gs://cloud-samples-data/generative-ai/video/ad_copy_from_video.mp4",
            mime_type="video/mp4",
        ),
        "What is in the video?",
    ],
)
print(response.text)

### Code Execution

The Gemini API code execution feature enables the model to generate and run Python code and learn iteratively from the results until it arrives at a final output.